In [35]:
import requests
import pandas as pd

from dotenv import load_dotenv
load_dotenv()

import os

In [36]:
TRAFIKVERKET_API=os.getenv("TRAFIKVERKET_KEY")
TRAFIKVERKET_URL="https://api.trafikinfo.trafikverket.se/v2/data.json"

In [37]:
question = f"""<REQUEST>
  <LOGIN authenticationkey="{TRAFIKVERKET_API}"/>
  <QUERY objecttype="TrainAnnouncement" schemaversion="1.9" limit="10">
    <FILTER>
      <EQ name="AdvertisedTrainIdent" value="936" />
      <GT name="AdvertisedTimeAtLocation" value="2024-04-19T00:00:00.000+02:00" />
      <LT name="AdvertisedTimeAtLocation" value="2024-04-20T00:00:00.000+02:00" />
    </FILTER>
  </QUERY>
</REQUEST>"""

question

'<REQUEST>\n  <LOGIN authenticationkey="e4f6b2548ecc4400b9fc59c4a4710fe8"/>\n  <QUERY objecttype="TrainAnnouncement" schemaversion="1.9" limit="10">\n    <FILTER>\n      <EQ name="AdvertisedTrainIdent" value="936" />\n      <GT name="AdvertisedTimeAtLocation" value="2024-04-19T00:00:00.000+02:00" />\n      <LT name="AdvertisedTimeAtLocation" value="2024-04-20T00:00:00.000+02:00" />\n    </FILTER>\n  </QUERY>\n</REQUEST>'

In [38]:
trainstation_question = f"""<REQUEST>
  <LOGIN authenticationkey="{TRAFIKVERKET_API}"/>
  <QUERY objecttype="TrainAnnouncement" schemaversion="1.9" limit="10">
    <FILTER>
      <EQ name="LocationSignature" value="Cst" />
      <GT name="AdvertisedTimeAtLocation" value="$dateadd(-1)" />
      <LT name="AdvertisedTimeAtLocation" value="$now" />
    </FILTER>
  </QUERY>
</REQUEST>"""

trainstation_question

'<REQUEST>\n  <LOGIN authenticationkey="e4f6b2548ecc4400b9fc59c4a4710fe8"/>\n  <QUERY objecttype="TrainAnnouncement" schemaversion="1.9" limit="10">\n    <FILTER>\n      <EQ name="LocationSignature" value="Cst" />\n      <GT name="AdvertisedTimeAtLocation" value="$dateadd(-1)" />\n      <LT name="AdvertisedTimeAtLocation" value="$now" />\n    </FILTER>\n  </QUERY>\n</REQUEST>'

In [39]:
headers = {
  "Content-Type": "text/xml"
}
resp = requests.post(TRAFIKVERKET_URL, data=trainstation_question, headers=headers)
# resp = requests.post(TRAFIKVERKET_URL, data=trainstation_question)

resp

<Response [200]>

In [43]:
traininfo = resp.json()

traininfo

{'RESPONSE': {'RESULT': [{'TrainAnnouncement': [{'ActivityId': '1500adde-f75d-c409-08dc-5438b3b08f53',
      'ActivityType': 'Ankomst',
      'Advertised': True,
      'AdvertisedTimeAtLocation': '2024-04-19T14:15:00.000+02:00',
      'AdvertisedTrainIdent': '19',
      'Canceled': False,
      'Deleted': False,
      'DepartureDateOTN': '2024-04-19T00:00:00.000+02:00',
      'EstimatedTimeIsPreliminary': False,
      'FromLocation': [{'LocationName': 'Fln', 'Priority': 1, 'Order': 0}],
      'InformationOwner': 'SJ',
      'LocationDateTimeOTN': '2024-04-19T14:15:00.000+02:00',
      'LocationSignature': 'Cst',
      'MobileWebLink': 'http://mobil.sj.se',
      'ModifiedTime': '2024-04-19T12:17:07.917Z',
      'NewEquipment': 0,
      'Operator': 'SJ',
      'OperationalTrainNumber': '10019',
      'OperationalTransportIdentifiers': [{'ObjectType': 'TR',
        'Company': '1174',
        'Core': '----------19',
        'Variant': '02',
        'TimetableYear': 2024,
        'StartDat

In [41]:
df = pd.DataFrame(traininfo["RESPONSE"]["RESULT"][0]["TrainAnnouncement"])

df.head(1)

,ActivityId,ActivityType,Advertised,AdvertisedTimeAtLocation,AdvertisedTrainIdent,Canceled,Deleted,DepartureDateOTN,EstimatedTimeIsPreliminary,FromLocation,...,ToLocation,TrackAtLocation,TrainOwner,TypeOfTraffic,ViaFromLocation,WebLink,WebLinkName,Booking,ViaToLocation,TrainComposition
0,1500adde-f75d-c409-08dc-5438b3b08f53,Ankomst,True,2024-04-19T14:15:00.000+02:00,19,False,False,2024-04-19T00:00:00.000+02:00,False,"[{'LocationName': 'Fln', 'Priority': 1, 'Order...",...,"[{'LocationName': 'Cst', 'Priority': 1, 'Order...",10,SJ,"[{'Code': 'YNA001', 'Description': 'Tåg'}]","[{'LocationName': 'Blgc', 'Priority': 1, 'Orde...",http://www.sj.se/trafikinfo,SJ,NaN,NaN,NaN


In [50]:
df = df[['AdvertisedTimeAtLocation', 'TimeAtLocation']]

df['AdvertisedTimeAtLocation'] = pd.to_datetime(df['AdvertisedTimeAtLocation'])
df['TimeAtLocation'] = pd.to_datetime(df['TimeAtLocation'])

df['Delay'] = df['TimeAtLocation'] - df['AdvertisedTimeAtLocation']

df.mean()

AdvertisedTimeAtLocation    2024-04-19 14:10:06+02:00
TimeAtLocation              2024-04-19 14:09:42+02:00
Delay                               -1 days +23:59:36
dtype: object

In [ ]:
df.plot()